# Lyrics.com artist classifier




**Goals:**  
In this project, we will build a text classification model on song lyrics. 
The task is to predict the artist from a piece of text. 
To train such a model, you first need to collect your own lyrics dataset:

- Download a HTML page with links to songs

- Extract hyperlinks of song pages

- Download and extract the song lyrics

- Vectorize the text using the Bag Of Words method

- train a classification model that predicts the artist from a piece of text

- refactor the code into functions

- Write a simple command-line interface for the program

- upload your code to GitHub



## Lyrics download

- dev functions that:
    1. takes artist name, checks if exists, 
    2. grabs all lyrics links
    3. downloads lyrics

### Artist Checking

In [1]:
import lyrics_fun as lf

In [2]:
arep = lf.get_artist("Mô", True)
arep

Artist: Mô ----- Status: 200
    Several options for artist; picking first


{'base_url': 'https://www.lyrics.com',
 'artist': 'Mô',
 'url': 'https://www.lyrics.com/artist/M%C3%B4/',
 'url_refined': 'https://www.lyrics.com/artist/Mo/106563',
 'response': <Response [200]>,
 'status_code': 200,
 'exists_on_site': True}

In [3]:
arep = lf.get_artist("twiddieyiszl", True)
arep

Artist: twiddieyiszl ----- Status: 200
    No unique artist found


{'base_url': 'https://www.lyrics.com',
 'artist': 'twiddieyiszl',
 'url': 'https://www.lyrics.com/artist/twiddieyiszl/',
 'url_refined': None,
 'response': <Response [200]>,
 'status_code': 200,
 'exists_on_site': False}

In [4]:
# test with single hit
req = lf.get_artist("Olivia Rodrigo", True)
arep = req['response']
aurl =req['url']
lf.refine_artist_link(arep, aurl, 'lyrics.com/', False)


Artist: Olivia Rodrigo ----- Status: 200
    Found unique artist


'https://www.lyrics.com/artist/Olivia%20Rodrigo/'

In [5]:
# test with single hit
req = lf.get_artist("Kiss", False)
arep = req['response']
aurl =req['url']
lf.refine_artist_link(arep, aurl, 'lyrics.com/', True)


    Found unique artist


'https://www.lyrics.com/artist/Kiss/'

In [6]:
# test with multi hit
req = lf.get_artist("sun", True)
arep = req['response']
aurl =req['url']
lf.refine_artist_link(arep, aurl, 'lyrics.com/', True)


Artist: sun ----- Status: 200
    Several options for artist; picking first
    Several options for artist; picking first


'lyrics.com//artist/Sun/5559'

In [7]:
# test with no hit
req = lf.get_artist("gibidigbisij2", True)
arep = req['response']
aurl =req['url']
lf.refine_artist_link(arep, aurl, 'lyrics.com/', True)


Artist: gibidigbisij2 ----- Status: 200
    No unique artist found
    No unique artist found


In [8]:
artist = lf.get_artist("Olivia Rodrigo", True)
artist

Artist: Olivia Rodrigo ----- Status: 200
    Found unique artist


{'base_url': 'https://www.lyrics.com',
 'artist': 'Olivia Rodrigo',
 'url': 'https://www.lyrics.com/artist/Olivia%20Rodrigo/',
 'url_refined': 'https://www.lyrics.com/artist/Olivia%20Rodrigo/',
 'response': <Response [200]>,
 'status_code': 200,
 'exists_on_site': True}

### Grab lyric links


- check if `exists_on_site = True`
- grab all song links

In [1]:
import lyrics_fun as lf

artist = lf.get_artist("Sun", True)
artist_links = lf.extract_lyric_links(artist, drop_duplicates = True, drop_instrumentals = True, drop_similar=True, verbose=True)
len(artist_links['links'])

Artist: Sun ----- Status: 200
    Several options for artist; picking first
    Dropped 20 duplicated lyric links
    Dropped 1 instrumental lyric link(s)
    Dropped 0 Remix/Acoustic lyric link(s)


6

In [10]:
artist

{'base_url': 'https://www.lyrics.com',
 'artist': 'Sun',
 'url': 'https://www.lyrics.com/artist/Sun/',
 'url_refined': 'https://www.lyrics.com/artist/Sun/5559',
 'response': <Response [200]>,
 'status_code': 200,
 'exists_on_site': True}

In [11]:
artist_links

{'base_url': 'https://www.lyrics.com',
 'artist': 'Sun',
 'url': 'https://www.lyrics.com/artist/Sun/',
 'url_refined': 'https://www.lyrics.com/artist/Sun/5559',
 'response': <Response [200]>,
 'status_code': 200,
 'exists_on_site': True,
 'links': ['https://www.lyrics.com/lyric/7067990/Sun/I+Had+a+Choice',
  'https://www.lyrics.com/lyric/3611604/Sun/Sun+Is+Here',
  'https://www.lyrics.com/lyric/4595999/Sun/Force+of+Nature',
  'https://www.lyrics.com/lyric/4596655/Sun/My+Woman',
  'https://www.lyrics.com/lyric-lf/6468010/Sun/Sailor',
  'https://www.lyrics.com/lyric-lf/1361310/Sun/Ms.+Communication']}

### Download lyrics

Options for dropping duplicates and instrumentals

In [12]:
extract_test = lf.extract_lyric(artist_links=artist_links, verbose = False)

In [14]:
lf.pd.DataFrame({'title' : extract_test['lyric_title'], 'lyric': extract_test['lyric_text']})

,title,lyric
0,I Had a Choice,"Holding hands, strolling through the park Swee..."
1,Sun Is Here,You come in numbers to feel a groove I have a ...
2,Force of Nature,It's gettin kinda heavy! Yeah yeah It's gettin...
3,My Woman,Little boy was blue because he knew that soon ...
4,Sailor,Sailor What do you want from me? Sailor I'm no...
5,Ms. Communication,I just wanna put it all on the table Maybe we ...


In [1]:
import lyrics_fun as lf


fail_artist = lf.get_artist("gibidigbisij2", True)
fail_links = lf.extract_lyric_links(fail_artist, drop_duplicates = True, drop_instrumentals = True, drop_similar=True, verbose=True)


lf.extract_lyric(artist_links=fail_links, verbose = False)




Artist: gibidigbisij2 ----- Status: 200
    No unique artist found
    Can only proceed with an artist listed at lyrics.com, ensure exists_on_site is True. Returning None
    Can only proceed with an artist listed at lyrics.com, ensure lyric link extraction was successful. Returning None


## Improvements

- Scan for non-english text?! (found korean symbols in one song from Sun)
- Strip symbols/punctuation/notation
- strip typical lyrical notation/syntax, like "Verse 2", "Hook"\
- **implement max_lyrics** in extract_lyrics() with a reasonable default value of e.g., 50
- ~~implement `drop_similar` and use regex to search for patterns: "acoustic|remix"~~ (needs more testing)
- refactor code for duplicates/similar/instrumental to:
    if any(drop_instrumental, drop_remix, drop_acoustic)):
        - filter_for(*args, 'instrumental', 'acoustic', 'remix')
        - could use str.join("|", *args)


## Next Steps:

- ~~dev wrapper function to run all lyric steps~~
- dev wrapper to fit logReg model
    - make X_y df (wrapper for pd.DataFrame)
- dev wrapper with CLI

In [1]:
import lyrics_fun as lf
first = lf.process_artist(artist = "A Constant Knowledge of Death", verbose = True)
second = lf.process_artist(artist = "Y Teezy", verbose = True)


Artist: A Constant Knowledge of Death ----- Status: 200
    Several options for artist; picking first
    Dropped 0 duplicated lyric links
    Dropped 0 instrumental lyric link(s)
    Dropped 0 Remix/Acoustic lyric link(s)


In [3]:
artists = [first, second]

In [5]:
import pickle
with open('artist_test.pkl', 'wb') as atestpkl:
    pickle.dump(artists, atestpkl)

In [1]:
import lyrics_fun as lf
import pickle

with open('artist_test.pkl', 'rb') as atestpkl:
    artists = pickle.load(atestpkl)


model_df = lf.make_model_df([artists[0], artists[1]])
model_df

,artist,text
0,A Constant Knowledge of Death,Breathe Festering lungs Deprived of oxygen Syc...
1,A Constant Knowledge of Death,Enter the delusion Paths become limitless The ...
2,A Constant Knowledge of Death,"""And on your deathbed You inexplicably said 'T..."
3,A Constant Knowledge of Death,I think about death often It's not that it sca...
4,A Constant Knowledge of Death,Each year These walls More ravenous Ten percen...
...,...,...
29,Y Teezy,Lil Bitch Lil Bitch Lil Bitch Lil Bitch Say ev...
30,Y Teezy,Well Alright Let me tell yo ass bout Tommy my ...
31,Y Teezy,Papamitrou Boy Let me tell you something you a...
32,Y Teezy,Sunup to sundown I gotta get it Sunup to sundo...


In [2]:
xdf = lf.run_logreg(moddf=model_df, new_lyric='gang gun boy ass tommy star bitch sunup sundown')
xdf

1


{'predicted_artist': 'Y Teezy', 'predicted_proba': 0.5871213598767744}

In [4]:
any([len('a') > 0, len('b') > 1])

True

In [5]:
ls = [5,6,7,100,9]

In [11]:
ls.index(100)

3